<a href="https://colab.research.google.com/github/banatfatima/project/blob/main/Nakheel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3

def scrape_nakheel_listings(url):
    response = requests.get(url)
    html_content = response.content

    soup = BeautifulSoup(html_content, 'html.parser')

    listings = soup.select('.property-card')

    scraped_listings = []

    for listing in listings:
        property_title = listing.select_one('.property-title').get_text()
        property_price = listing.select_one('.property-price').get_text()
        property_address = listing.select_one('.property-location').get_text()

        scraped_listings.append({
            'title': property_title.strip(),
            'price': property_price.strip(),
            'address': property_address.strip()
        })

    return scraped_listings

def clean_data(listings):
    cleaned_listings = []

    for listing in listings:
        title = re.sub(r'<[^>]+>', '', listing['title']).strip()
        price = re.sub(r'[^\d.]', '', listing['price']).strip()
        address = listing['address'].strip()

        cleaned_listings.append({
            'title': title,
            'price': price,
            'address': address
        })

    return cleaned_listings

def store_in_database(listings):
    connection = sqlite3.connect('nakheel_listings.db')
    cursor = connection.cursor()

    cursor.execute('CREATE TABLE IF NOT EXISTS nakheel_listings ('
                   'Id INTEGER PRIMARY KEY AUTOINCREMENT,'
                   'Title TEXT,'
                   'Price TEXT,'
                   'Address TEXT'
                   ')')
    connection.commit()

    for listing in listings:
        cursor.execute('INSERT INTO nakheel_listings (Title, Price, Address) VALUES (?, ?, ?)',
                       (listing['title'], listing['price'], listing['address']))
        connection.commit()

    connection.close()

def main():
    nakheel_url = 'https://www.nakheel.com/en/properties-for-sale'
    scraped_listings = scrape_nakheel_listings(nakheel_url)
    cleaned_listings = clean_data(scraped_listings)
    store_in_database(cleaned_listings)

    # Print the cleaned listings (optional)
    print(cleaned_listings)

if __name__ == "__main__":
    main()


[]
